### Part a

In [1]:
import symforce
symforce.set_symbolic_api("symengine")
symforce.set_epsilon_to_symbol()
import symforce.symbolic as sf
from symforce.notebook_util import display
from symforce.notebook_util import print_expression_tree

x, y, z, fx, cx, fy, cy = sf.symbols('x y z fx cx fy cy')
f = sf.Matrix(
    [
        fx*x/z + cx, 
        fy*y/z + cy
    ]
)

display(f.jacobian([x,y,z]))

⎡fx      -fx⋅x ⎤
⎢──  0   ──────⎥
⎢z          2  ⎥
⎢          z   ⎥
⎢              ⎥
⎢    fy  -fy⋅y ⎥
⎢0   ──  ──────⎥
⎢    z      2  ⎥
⎣          z   ⎦

### Part b

In [3]:
import numpy as np
from symforce.values import Values

num_poses = 3
num_landmarks = 3

initial_values = Values(
    poses=[sf.Pose2.identity()] * num_poses,
    landmarks=[sf.V2(-2, 2), sf.V2(1, -3), sf.V2(5, 2)],
    distances=[1.7, 1.4],
    angles=np.deg2rad([[145, 335, 55], [185, 310, 70], [215, 310, 70]]).tolist(),
    epsilon=sf.numeric_epsilon,
)

def bearing_residual(
    pose: sf.Pose2, landmark: sf.V2, angle: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    t_body = pose.inverse() * landmark
    predicted_angle = sf.atan2(t_body[1], t_body[0], epsilon=epsilon)
    return sf.V1(sf.wrap_angle(predicted_angle - angle))

def odometry_residual(
    pose_a: sf.Pose2, pose_b: sf.Pose2, dist: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    return sf.V1((pose_b.t - pose_a.t).norm(epsilon=epsilon) - dist)

from symforce.opt.factor import Factor

factors = []

# Bearing factors
for i in range(num_poses):
    for j in range(num_landmarks):
        factors.append(Factor(
            residual=bearing_residual,
            keys=[f"poses[{i}]", f"landmarks[{j}]", f"angles[{i}][{j}]", "epsilon"],
        ))

# Odometry factors
for i in range(num_poses - 1):
    factors.append(Factor(
        residual=odometry_residual,
        keys=[f"poses[{i}]", f"poses[{i + 1}]", f"distances[{i}]", "epsilon"],
    ))